## All-sky NOIRLab Source Catalog (DR2)

To install NOAO datalab access:  
```pip install --ignore-installed --no-cache-dir noaodatalab```

Login via the command line to datalab:  
```datalab login``` 


Information on the columns each of tables contains can be found here:  
[Exposure](https://datalab.noirlab.edu/query.php?name=nsc_dr2.exposure)  
[Measurements](https://datalab.noirlab.edu/query.php?name=nsc_dr2.meas)  
[Coverage](https://datalab.noirlab.edu/query.php?name=nsc_dr2.coverage)  
[Chip](https://datalab.noirlab.edu/query.php?name=nsc_dr2.chip)  
[Object](https://datalab.noirlab.edu/query.php?name=nsc_dr2.object)  



**References**:  
Nidever, D. L., Dey, A., Fasbender, K., Juneau, S., Meisner, A. M., Wishart, J., Scott, A., Matt, K., Nikutta, R., & Pucha, R. (2021). Second Data Release of the All-sky NOIRLab Source Catalog. The Astronomical Journal, 161(4), 192. https://doi.org/10.3847/1538-3881/ABD6E1

In [ ]:
import os
import pandas as pd
import numpy as np
import healpy as hp 
import matplotlib.pyplot as plt

%matplotlib inline 

from astropy.time import Time

In [ ]:
DATA_DIR = "/mnt/data/projects/thor/thor_data/nsc/"

In [ ]:
from dl import authClient as ac
from dl import queryClient as qc
from getpass import getpass

username = 'moeyensj'
token = ac.login(username,getpass('Account Password: '))
if not ac.isValidToken(token):
    print('Error: invalid login for user %s (%s)' % (username,token))
else:
    print("Login token:   %s" % token)

Get the contents of the `exposure` table.

In [ ]:
file_name = os.path.join(DATA_DIR, "nsc_dr2_exposure.csv")
if not os.path.exists(file_name):
    query = """SELECT * FROM nsc_dr2.exposure ORDER BY mjd ASC"""
    result = qc.query(token, sql=query, fmt='csv', out=file_name)
exposures = pd.read_csv(file_name, index_col=False)

In [ ]:
exposures

In [ ]:
file_name = os.path.join(DATA_DIR, "nsc_dr2_coverage.csv")
if not os.path.exists(file_name):
    query = """SELECT * FROM nsc_dr2.coverage LIMIT 100"""
    result = qc.query(token, sql=query, fmt='csv', out=file_name)
coverage = pd.read_csv(file_name, index_col=False)

In [ ]:
coverage

The NSC DR2 release contains data from three instruments:

In [ ]:
exposures["instrument"].unique()

CTIO-4m+DECam : c4d  
KPNO-4m+Mosaic3 : ksb  
Bok-2.3+90Prime : k4m  

Number of measurements per instrument:

In [ ]:
exposures.groupby(by="instrument")["nmeas"].sum()

In [ ]:
exposures.groupby(by="instrument")["mjd"].apply(lambda x : x.max() - x.min())

In [ ]:
exposures.groupby(by="instrument")["mjd"].describe()

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=200)
for instrument in exposures["instrument"].unique():
    mask = exposures["instrument"].isin([instrument])

    ax.scatter(
        exposures[mask]["mjd"].values, 
        exposures[mask]["nmeas"].values, 
        label=instrument,
        s=1
    )
ax.set_yscale("log")
ax.legend(
    frameon=False,
    bbox_to_anchor=(1.03, 0.6)
)
ax.set_ylabel("Measurements")
ax.set_xlabel("Observation Time [MJD]")

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=200)
for instrument in exposures["instrument"].unique():
    mask = exposures["instrument"].isin([instrument])

    ax.scatter(
        exposures[mask]["mjd"].values, 
        exposures[mask]["depth95"].values, 
        label=instrument,
        s=1
    )
ax.legend(
    frameon=False,
    bbox_to_anchor=(1.03, 0.6)
)
ax.set_ylabel(r"Depth $P_{95}$")
ax.set_xlabel("Observation Time [MJD]")

In [ ]:
start_mjd = 56515
window_size = 20
window = exposures[
    (exposures["instrument"] == "c4d") 
    & (exposures["mjd"] <= start_mjd + window_size)
    & (exposures["mjd"] >= start_mjd)
]

In [ ]:
np.round(window["mjd"].max(), 2), np.round(window["mjd"].min() - 0.01, 2)


In [ ]:
query = """
SELECT COUNT(*)
FROM nsc_dr2.object 
WHERE (ndet <= 4)
"""
results = qc.query(token, adql=query, fmt='pandas', timeout=3600, async_=True, wait=True, poll=10, verbose=1)
results

In [ ]:
query = """
SELECT COUNT(*)
FROM nsc_dr2.object 
WHERE (ndet <= 4)
"""
results = qc.query(token, adql=query, fmt='pandas', timeout=3600, async_=True, wait=True, poll=10, verbose=1)
results

Number of objects with ndet<=4: 1763381575  
Number of objects with ndet==1: 886983556 

In [ ]:
ras = np.linspace(0, 360, 360 * 10 + 1)

In [ ]:
def queryRASlice(ra_start, ra_end):
    
    #print(f"Processesing {ra_start:06.2f}<=RA<{ra_end:06.2f}")
    file_name = os.path.join(DATA_DIR, f"nsc_dr2_observations_{ra_start:06.2f}_{ra_end:06.2f}.csv")
    
    if not os.path.exists(file_name):
        query = f"""
        SELECT o.id, o.ra AS mean_ra, o.dec AS mean_dec, o.ndet, o.nphot, o.mjd AS mean_mjd, o.deltamjd, m.measid, m.mjd, m.ra, m.dec, m.raerr, m.decerr, m.mag_auto, m.magerr_auto, m.filter, m.exposure, m.class_star 
        FROM nsc_dr2.object AS o 
        JOIN nsc_dr2.meas as m 
        ON o.id = m.objectId 
        WHERE (o.ndet <= 4) AND (o.ra >= {ra_start}) AND (o.ra < {ra_end})
        """
        results = qc.query(token, adql=query, fmt='pandas', timeout=3600, async_=True, wait=True, poll=5, verbose=0)
        results.sort_values(by=["mjd", "measid"], inplace=True, ascending=[True, True])
        results.to_csv(file_name, index=False)
        
    else:
        #print("Already processed.")
        pass
        
    return

In [ ]:
import multiprocessing as mp

pool = mp.Pool(10)
pool.starmap(
    queryRASlice,
    zip(ras[:-1][:20], ras[1:][:20])
)
pool.close()

In [ ]:
phi = np.radians(preprocessed_observations["RA_deg"].values)
theta = np.radians(90 - preprocessed_observations["Dec_deg"].values)
mjd_utc = preprocessed_observations["mjd_utc"].values


nside = 32
print(np.degrees(hp.nside2resol(nside)))
pixel_indices = hp.ang2pix(nside, theta, phi)
vals, counts = np.unique(pixel_indices, return_counts=True)
preprocessed_observations["pixel_ind"] = pixel_indices
visits = preprocessed_observations.groupby(by=["pixel_ind"])["mjd_utc"].nunique()
visits = visits[visits >= 5]

num_pix = hp.nside2npix(nside)
m = hp.ma(np.zeros(num_pix))
m[visits.index.values] = visits.values
m.mask = np.ones(len(m)) 
m.mask[visits.index.values] = False 